In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [2]:
# 데이터를 불러온다.
df1 = pd.read_csv('./data/ThoraricSurgery.csv', header = None)
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76,0
466,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77,0
467,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62,0
468,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58,1


In [3]:
# 입력과 결과로 나눈다.
X = df1.drop(17, axis=1)
y = df1[17]

display(X)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76
466,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77
467,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62
468,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58


0      0
1      0
2      1
3      1
4      0
      ..
465    0
466    0
467    0
468    1
469    0
Name: 17, Length: 470, dtype: int64

In [4]:
# 신경망을 설계한다.
# 신경망 모델을 관리하는 객체를 생성한다
# 여기에 층을 쌓아준다.
model = Sequential()

# 신경망은 입력층 => 은닉층들 => 출력층으로 구성된다.

# 첫 번째 은닉층
# 첫 번째 은닉층은 input_dim은 입력층의 노드의 개수가 된다.
# 입력데이터의 컬럼 수와 동일하게 작성해줘야 한다.

# Dense : 선형 회귀를 수행하는 layer
# 첫 번째 숫자는 은닉층의 노드의 개수가 된다.
model.add(Dense(30, input_dim=17))
model.add(Activation('relu'))

# 마지막에 설정한 레이어는 무조건 출력층이다.
# Dense의 1은 출력 데이터의 종류 수가 된다.
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [5]:
# 신경망 모델을 컨파일
# loss : 학습시 수행하는 평가때 사용할 손실함수
# opimizer : loss에서 설정한 함수를 통해 구한 손실값을 기반으로
# 신경망 모델을 보정하는 함수
# metrics : 추가로 평가하고자 하는 평가 기준을 설정한다.
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                540       
                                                                 
 activation (Activation)     (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
 activation_1 (Activation)   (None, 1)                 0         
                                                                 
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 학습
# eqcohs : 학습 횟수. 학습 횟수가 너무 많으면 과적합된다.
# 무조건 많이 주세요 ~~~ 나중에 중단 시킬 수 있습니다!!!!
# batch_size : 메모리에 할번에 올릴 데이터의 양(행의 개수)
# 적게 주면 메모리에 여유가 있지만 오래걸린다.
# 적당히... 무조건 많이 주시고 오류나면 낮추세요~
model.fit(X, y, epochs=30, batch_size=10)

Epoch 1/30
47/47 [==============================] - 1s 2ms/step - loss: 0.2462 - accuracy: 0.7362
Epoch 2/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1468 - accuracy: 0.8511
Epoch 3/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1463 - accuracy: 0.8511
Epoch 4/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1467 - accuracy: 0.8511
Epoch 5/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1457 - accuracy: 0.8511
Epoch 6/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1459 - accuracy: 0.8532
Epoch 7/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1454 - accuracy: 0.8511
Epoch 8/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1447 - accuracy: 0.8511
Epoch 9/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1454 - accuracy: 0.8511
Epoch 10/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1465 - accuracy: 0.8489
Epoch 11/30
47/47 [

In [7]:
# 예측한다.
pred1 = model.predict(X)
pred1

15/15 [==============================] - 0s 1ms/step


array([[1.02199023e-14],
       [4.06070709e-01],
       [4.66268480e-01],
       [3.72808665e-01],
       [2.67239988e-01],
       [1.61080346e-01],
       [8.93967301e-02],
       [3.37798268e-01],
       [5.51191680e-02],
       [8.24920426e-05],
       [2.54221686e-05],
       [8.45225659e-05],
       [4.03726881e-06],
       [2.20507427e-05],
       [4.39845032e-08],
       [2.90596222e-06],
       [2.16498984e-06],
       [3.10930409e-06],
       [1.81735587e-07],
       [9.22507510e-08],
       [8.37457179e-08],
       [3.13504813e-08],
       [3.01523997e-08],
       [7.24561799e-10],
       [5.04015107e-10],
       [3.03757020e-10],
       [1.81788543e-10],
       [7.01910127e-11],
       [8.36381658e-11],
       [9.26902739e-12],
       [1.58507430e-13],
       [3.90815039e-14],
       [1.37275123e-14],
       [3.37245511e-15],
       [1.21616853e-15],
       [3.08324361e-19],
       [1.97655985e-16],
       [7.28290509e-17],
       [1.14461719e-16],
       [4.24115326e-16],


In [8]:
# 0.5 초과는 1 아닌 것은 0으로 환산한다.
result1 = (pred1 > 0.5).astype(int)
result1

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [9]:
# 정확도 확인
score1 = accuracy_score(y, result1)
score1

0.8531914893617021